In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Paths and basic settings
data_dir = os.path.abspath(".")  # current directory holding the THAI* folders
img_size = (224, 224)
batch_size = 32
seed = 42
epochs = 20  # adjust as needed

# Load train/val splits from folders
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    labels="inferred",
    shuffle=True,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    labels="inferred",
    shuffle=True,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

# Data augmentation and preprocessing
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

preprocess_input = tf.keras.applications.resnet50.preprocess_input

# Build model with transfer learning
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=img_size + (3,),
)
base_model.trainable = False  # freeze base for initial training

inputs = keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()

# Callbacks
checkpoint_path = "resnet50_thai_notes.keras"
callbacks = [
    keras.callbacks.ModelCheckpoint(
        checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy", patience=5, restore_best_weights=True
    ),
]

# Train (frozen base)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks,
)

# Fine-tune: unfreeze top layers of the base model (optional)
# fine_tune_at = 140  # unfreeze from this layer index upward; adjust as needed
# base_model.trainable = True
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False

# model.compile(
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#     loss="sparse_categorical_crossentropy",
#     metrics=["accuracy"],
# )

# fine_tune_epochs = 10
# total_epochs = epochs + fine_tune_epochs

# history_fine = model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=total_epochs,
#     initial_epoch=history.epoch[-1] + 1,
#     callbacks=callbacks,
# )

# Evaluate and save final model
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc:.4f}")

model.save("resnet50_thai_notes_final.keras")

Found 800 files belonging to 5 classes.
Using 640 files for training.
Found 800 files belonging to 5 classes.
Using 160 files for validation.
Classes: ['THAI100', 'THAI1000', 'THAI20', 'THAI50', 'THAI500']


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 224, 224)  │          0 │ sequential_1[0][… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 224, 224)  │          0 │ sequential_1[0][… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 224, 224)  │          0 │ sequential_1[0][… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack_1 (Stack)     │ (None, 224, 224,  │          0 │ get_item_3[0][0], │
│                     │ 3)                │            │ get_item_4[0][0], │
│                     │                   │            │ get_item_5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 224, 224,  │          0 │ stack_1[0][0]     │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add_1[0][0]       │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 2048)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5)         │     10,245 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,597,957 (90.02 MB)

 Trainable params: 10,245 (40.02 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 966ms/step - accuracy: 0.2543 - loss: 1.9647
Epoch 1: val_accuracy improved from -inf to 0.45625, saving model to resnet50_thai_notes.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.2567 - loss: 1.9553 - val_accuracy: 0.4563 - val_loss: 1.2982
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 959ms/step - accuracy: 0.4993 - loss: 1.1805
Epoch 2: val_accuracy improved from 0.45625 to 0.68125, saving model to resnet50_thai_notes.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5021 - loss: 1.1764 - val_accuracy: 0.6812 - val_loss: 0.9091
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7194 - loss: 0.7996
Epoch 3: val_accuracy improved from 0.68125 to 0.76250, saving model to resnet50_thai_notes.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7209 - loss: 0.7979 - val_accuracy: 0.7625 - val_loss: 0.7204
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 939ms/step - accuracy: 0.8358 - loss: 0.5876
Epoch 4: val_accuracy i

NameError: name 'model' is not defined